# Decomposição LU

A idéia da decomposição LU é criar 2 matrizes auxiliares:

$A = LU$

Assim, a equação $Ax = b$ pode ser reescrita como:

$LUx = b$

Fazendo:

$Ux = y$

temos:

$Ly = b$

Na decomposição LU, buscamos uma matriz $U$ que seja triangular superior:

\begin{bmatrix}
u_{11} & u_{12} & \ldots & u_{1n} \\
0 & u_{22} & \ldots & u_{2n} \\
\vdots \\
0 & 0 & \ldots & u_{1n} \\
\end{bmatrix}

e uma matriz $L$ que seja triangular inferior, com 1 na diagonal principal:

\begin{bmatrix}
1 & 0 & \ldots & 0 \\
l_{21} & 1 & \ldots & 0 \\
\vdots \\
l_{n1} & l_{n2} & \ldots & 1 \\
\end{bmatrix}

Em seguida, resolvemos o sistema $Ly = b$, achando o valor de $y$.

Depois, resolvemos o sistema $Ux = y$ e achamos o valor de $x$.

Qual a vantagem de utilizar a decomposição LU?

* Computacionalmente mais eficiente (quando aplicado para vários vetores $b$, usando a mesma matriz $A$)
* Cálculo da inversa de uma matriz

## Exemplo:

Resolva o seguinte sistema de equações:

\begin{aligned}
\begin{bmatrix}
1 & 1 & 1 \\
2 & 1 & -1 \\
2 & -1 & 1 
\end{bmatrix}
\begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} = 
\begin{bmatrix} b_1 \\ b_2 \\ b_3 \end{bmatrix}
\end{aligned}

para:

$ b = \begin{bmatrix} -2 \\ 1 \\ 3 \end{bmatrix}$

e

$ b = \begin{bmatrix} 1 \\ 1 \\ 3 \end{bmatrix}$

In [36]:
import numpy as np
import scipy.linalg

A = np.array([ [1, 1, 1], [2, 1, -1], [2, -1, 1] ])
    
LU, P = scipy.linalg.lu_factor(A)

b = np.array( [-2, 1, 3] )
x = scipy.linalg.lu_solve( (LU, P), b)
print(x)
print('Verificando a resposta:', A @ x, ' = ', b)

b = np.array( [1, 1, 3] )
x = scipy.linalg.lu_solve((LU, P), b)
print(x)
print('Verificando a resposta:', A @ x, ' = ', b)

[ 1. -2. -1.]
Verificando a resposta: [-2.  1.  3.]  =  [-2  1  3]
[ 1.  -0.5  0.5]
Verificando a resposta: [1. 1. 3.]  =  [1 1 3]


In [37]:
print('P: (vetor)\n', P)
print('LU:\n', LU)
P1,L,U = scipy.linalg.lu(A)
print('P:\n',P1)
print('L:\n', L)
print('U:\n', U)

P: (vetor)
 [1 2 2]
LU:
 [[ 2.    1.   -1.  ]
 [ 1.   -2.    2.  ]
 [ 0.5  -0.25  2.  ]]
P:
 [[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
L:
 [[ 1.    0.    0.  ]
 [ 1.    1.    0.  ]
 [ 0.5  -0.25  1.  ]]
U:
 [[ 2.  1. -1.]
 [ 0. -2.  2.]
 [ 0.  0.  2.]]


O método de eliminação de Gauss pode ser adaptado para produzir a decomposição LU.
Neste caso a matriz $U$ é a matriz obtida pela eliminação de Gauss (considerando apenas a matriz $A$) e a matriz $L$ é formada pelos fatores utilizados para subtrair as linhas, ou seja:

$l_{ij} = a'_{ij}/a'_{jj}$

In [38]:
import numpy as np
import scipy.linalg 

def lu(A):
    n = np.shape(A)[0]
    L = np.identity(n)
    U = A.copy()

    for PIVO in range(n):
        for LIN in range(PIVO+1, n):
            C = U[LIN,PIVO]/U[PIVO,PIVO]
            L[LIN,PIVO] = C
            U[LIN,:] = U[LIN,:] - C*U[PIVO,:]
    return L,U
    
# https://johnfoster.pge.utexas.edu/numerical-methods-book/LinearAlgebra_LU.html
def forward_substitution(L, b):
    #Get number of rows
    n = L.shape[0]
    
    #Allocating space for the solution vector
    y = np.zeros_like(b, dtype=np.double);
    
    #Here we perform the forward-substitution.  
    #Initializing  with the first row.
    y[0] = b[0] / L[0, 0]
    
    #Looping over rows in reverse (from the bottom  up),
    #starting with the second to last row, because  the 
    #last row solve was completed in the last step.
    for i in range(1, n):
        y[i] = (b[i] - np.dot(L[i,:i], y[:i])) / L[i,i]
        
    return y

def back_substitution(U, y):
    #Number of rows
    n = U.shape[0]
    
    #Allocating space for the solution vector
    x = np.zeros_like(y, dtype=np.double);

    #Here we perform the back-substitution.  
    #Initializing with the last row.
    x[-1] = y[-1] / U[-1, -1]
    
    #Looping over rows in reverse (from the bottom up), 
    #starting with the second to last row, because the 
    #last row solve was completed in the last step.
    for i in range(n-2, -1, -1):
        x[i] = (y[i] - np.dot(U[i,i:], x[i:])) / U[i,i]
        
    return x

A = np.array([[1,1,1], [2,1,-1], [2,-1,1]])
#A = np.array([[3, 2, 4], [1, 1, 2], [4, 3, -2]])

#P,L,U = scipy.linalg.lu(A)
#print('LU de A:\n')
L,U = lu(A)

#print(P, '\n')
print(L, '\n')
print(U, '\n')
#print(P @ L @ U)


[[1. 0. 0.]
 [2. 1. 0.]
 [2. 3. 1.]] 

[[ 1  1  1]
 [ 0 -1 -3]
 [ 0  0  8]] 



In [40]:
b = np.array([-2,1,3])
#b = np.array([1,1,3])

#y = forward_substitution(L, P.T @  b)
y = forward_substitution(L, b)
x = back_substitution(U, y)

print(A @ x)
#print(np.linalg.solve(A, b))

[-2.  1.  3.]
